In [1]:
import findspark
findspark.find()
findspark.init(edit_profile=True)
import pyspark
sc = pyspark.SparkContext()

##Clarification
When solving this problem, I referenced to the PageRank algorithm in PageRank.py

In [2]:
links = sc.textFile('/Users/Elena/Desktop/links-simple-sorted.txt', 4)
page_names = sc.textFile('/Users/Elena/Desktop/titles-sorted.txt', 4)

In [22]:
#HELPER FUNCTIONS
def construct_neighbor_graph(s):
    src, dests = s.split(': ')
    dests = [int(to) for to in dests.split(' ')]
    return (int(src), dests)


# create an RDD for looking up page names from numbers
# remember that it's all 1-indexed
def construct_name_dict(page_names):
    return construct_name_dict

def bfs_paths(graph, start, goal):
    queue = [(start, [start])]
    while queue:
        (vertex, path) = queue.pop(0)
        try:
            for next in graph[vertex] - set(path):
                if next == goal:
                    yield path + [next]
                else:
                    queue.append((next, path + [next]))
        except: 
            print vertex, path


def shortest_path(graph, start, goal):
    try:
        return next(bfs_paths(graph, start, goal))
    except StopIteration:
        return None

In [6]:
# construct and cache neighbor_graph and page_names (for lookup purpose)
neighbor_graph = links.map(construct_neighbor_graph).partitionBy(64).cache()
page_names = page_names.zipWithIndex().map(lambda (n, id): (id + 1, n)).sortByKey().cache()

In [23]:
# find Kevin Bacon
Kevin_Bacon = page_names.filter(lambda (K, V): V == 'Kevin_Bacon').collect()
# This should be [(node_id, 'Kevin_Bacon')]
assert len(Kevin_Bacon) == 1
Kevin_Bacon = Kevin_Bacon[0][0]  # extract node id

# find Harvard University
Harvard_University = page_names.filter(lambda (K, V):
                                       V == 'Harvard_University').collect()
# This should be [(node_id, 'Harvard_University')]
assert len(Harvard_University) == 1
Harvard_University = Harvard_University[0][0]  # extract node id

In [24]:
print Kevin_Bacon, Harvard_University

2729536 2152782


In [13]:
#Test Graph
neighbor_graph_simple = neighbor_graph.filter(lambda x: 
                                                  (Harvard_University == x[0]) or
                                                  (Harvard_University in x[1]) or
                                                  (Kevin_Bacon == x[0]) or
                                                  (Kevin_Bacon in x[1])
                                                 ).map(lambda x: dict({x[0]: set(x[1])})).collect()

# Solution 1: Naive

In [30]:
print 'Begin searching for Shortest Path from "Harvard University" to "Kevin Bacon": \n'
h_k_shortest_path = shortest_path(neighbor_graph_simple, Harvard_University, Kevin_Bacon)
h_k_shortest_path = sc.parallelize(h_k_shortest_path).map(lambda x: page_names.lookup(x)[0]).collect()
print 'Shortest Path from “Harvard University” to "Kevin Bacon": \n', h_k_shortest_path


print 'Begin searching for Shortest Path from "Kevin Bacon" to "Harvard University": \n'
k_h_shortest_path = shortest_path(neighbor_graph_simple, Kevin_Bacon, Harvard_University)
k_h_shortest_path = sc.parallelize(k_h_shortest_path).map(lambda x: page_names.lookup(x)[0]).collect()
print 'Shortest Path from "Kevin Bacon” to “Harvard University": \n', k_h_shortest_path

In [29]:
# neighbor_graph_simple[0]

In [32]:
def construct_neighbor_graph(s):
    src, dests = s.split(': ')
    dests = [int(to) for to in dests.split(' ')]
    return (int(src), dests)

def bfs_paths(graph, start, goal):
    queue = [(start, [start])]
    while queue:
        (vertex, path) = queue.pop(0)
        for next in graph[vertex] - set(path):
            if next == goal:
                yield path + [next]
            else:
                queue.append((next, path + [next]))


def shortest_path(graph, start, goal):
    try:
        return next(bfs_paths(graph, start, goal))
    except StopIteration:
        return None

if __name__ == '__main__':
#   sc = pyspark.SparkContext()
    #sc.setLogLevel('WARN')

    links = sc.textFile('/Users/Elena/Desktop/links-simple-sorted.txt', 4)
    page_names = sc.textFile('/Users/Elena/Desktop/titles-sorted.txt', 4)

    # construct and cache neighbor_graph and page_names (for lookup purpose)
    neighbor_graph = links.map(construct_neighbor_graph).partitionBy(64).cache()
    page_names = page_names.zipWithIndex().map(lambda (n, id): (id + 1, n)).sortByKey().cache()

    # neighbor_graph_simple is used for simple bfs search for shortest path
#     neighbor_graph_simple = neighbor_graph.map(lambda x: dict({x[0]: set(x[1])})).collect()

    # find Kevin Bacon
    Kevin_Bacon = page_names.filter(lambda (K, V): V == 'Kevin_Bacon').collect()
    # This should be [(node_id, 'Kevin_Bacon')]
    assert len(Kevin_Bacon) == 1
    Kevin_Bacon = Kevin_Bacon[0][0]  # extract node id

    # find Harvard University
    Harvard_University = page_names.filter(lambda (K, V):
                                           V == 'Harvard_University').collect()
    # This should be [(node_id, 'Harvard_University')]
    assert len(Harvard_University) == 1
    Harvard_University = Harvard_University[0][0]  # extract node id

    neighbor_graph_simple = neighbor_graph.filter(lambda x: 
                                                  (Harvard_University == x[0]) or
                                                  (Harvard_University in x[1]) or
                                                  (Kevin_Bacon == x[0]) or
                                                  (Kevin_Bacon in x[1])
                                                 ).map(lambda x: dict({x[0]: set(x[1])})).collect()

    print 'Begin searching for Shortest Path from "Harvard University" to "Kevin Bacon": \n'
    h_k_shortest_path = shortest_path(neighbor_graph_simple, Harvard_University, Kevin_Bacon)
    h_k_shortest_path = sc.parallelize(h_k_shortest_path).map(lambda x: page_names.lookup(x)[0]).collect()
    print 'Shortest Path from “Harvard University” to "Kevin Bacon": \n', h_k_shortest_path


    print 'Begin searching for Shortest Path from "Kevin Bacon" to "Harvard University": \n'
    k_h_shortest_path = shortest_path(neighbor_graph_simple, Kevin_Bacon, Harvard_University)
    k_h_shortest_path = sc.parallelize(k_h_shortest_path).map(lambda x: page_names.lookup(x)[0]).collect()
    print 'Shortest Path from "Kevin Bacon” to “Harvard University": \n', k_h_shortest_path



#try out

In [ ]:
graph = {'A': set(['B', 'C']),
         'B': set(['A', 'D', 'E']),
         'C': set(['A', 'F']),
         'D': set(['B']),
         'E': set(['B', 'F']),
         'F': set(['C', 'E'])}
list(bfs_paths(graph, 'A', 'F'))
shortest_path(graph, 'A', 'F') # ['A', 'C', 'F']